# Volume Generation
This notebook explains a method to create volume mesh for both bones and cartilages. 

# imports and user-defined properties

In [2]:
import numpy as np
import meshplot as mp
from pathlib import Path
import os
import igl
import matplotlib.pyplot as plt
import sys

sys.path.append('../')

import src

INPUT SUBJECT INFORMATION:
- model_id: choose between m1 -> m11 to select one of the dataset in the repository.
- full_model: choose "True" if you have a closed surface representation of the cartilages and "False" if implicit.
- i_dim, o_dim: the input and output dimension ( 'mm' = millimeters, 'm' = meters )
- o_format: the format you want the files to be save at ( '.obj' , '.stl' )

In [4]:
# model id
model_id = 'm1'  

# Are you cartilage geometries full or implicit models?
full_model = True

# dimensions 
i_dim  = 'mm'     
o_dim  = 'mm'

# file format and suffix
i_format = '.obj'
o_format = '.msh'

DIRECTORIES and PATHS:
- main_dir: locate this directory to where the libhip repository is cloned to.
- ftetwild_dir: locate this path to the 'build' folder of ftetwild.You can install ftetwild here: https://wildmeshing.github.io/ftetwild/ 
- i_dir:  locate this directory to the input bone surface mesh files.
- json_o_dir & ftet_o_dir = these folders will contain the mid-output results where the user will not need.
- o_dir:  locate this directory to where you want the final volume mesh of the pelvic girdle and the two legs to be saved at.
- config: the config file containes the subject specific properties.

In [5]:
main_dir     = Path('..')

# please change this directory to the build folder of ftetwild 
ftetwild_dir = Path.home()/ 'Documents'/ 'Github'/ 'fTetWild'/ 'build'

i_dir        = main_dir/ 'model_generation'/ 'cargen_output'/ model_id
json_o_dir   = main_dir/ 'model_generation'/ 'mid_outputs'/ 'json_output'  / model_id
ftet_o_dir   = main_dir/ 'model_generation'/ 'mid_outputs'/ 'ftet_output'  / model_id
nodal_o_dir  = main_dir/ 'model_generation'/ 'mid_outputs'/ 'nodal_output' / model_id
o_dir        = main_dir/ 'model_generation'/ 'volgen_output' / model_id
   
config_name  = model_id + '_config'
config_path  = str((main_dir/ 'config'/ config_name).with_suffix('.yml'))
config       = src.Config (config_path)

# Remove all files inside output directory if it exists, otherwise create it
if json_o_dir.is_dir():
    for file in json_o_dir.iterdir():
        if file.is_file():
            file.unlink()
else:
    json_o_dir.mkdir(exist_ok=False)
    
if ftet_o_dir.is_dir():
    for file in ftet_o_dir.iterdir():
        if file.is_file():
            file.unlink()
else:
    ftet_o_dir.mkdir(exist_ok=False)
    
if nodal_o_dir.is_dir():
    for file in nodal_o_dir.iterdir():
        if file.is_file():
            file.unlink()
else:
    nodal_o_dir.mkdir(exist_ok=False)
    
    
if o_dir.is_dir():
    for file in o_dir.iterdir():
        if file.is_file():
            file.unlink()
else:
    o_dir.mkdir(exist_ok=False)

In [4]:
# input paths - bones
i_bone_tag = model_id + '_cg_bone_'

i_sacrum  = i_bone_tag + 'sacrum'
i_lpelvis = i_bone_tag + 'lpelvis'  
i_rpelvis = i_bone_tag + 'rpelvis'
i_lfemur  = i_bone_tag + 'lfemur'
i_rfemur  = i_bone_tag + 'rfemur'

i_sacrum_path  = str((i_dir/ i_sacrum).with_suffix(i_format).resolve())
i_lpelvis_path = str((i_dir/ i_lpelvis).with_suffix(i_format).resolve())
i_rpelvis_path = str((i_dir/ i_rpelvis).with_suffix(i_format).resolve())
i_lfemur_path  = str((i_dir/ i_lfemur).with_suffix(i_format).resolve())
i_rfemur_path  = str((i_dir/ i_rfemur).with_suffix(i_format).resolve())

In [5]:
# input paths - joints
if full_model:
    i_joint_tag = model_id + '_cg_jnt_'
else:
    i_joint_tag = model_id + '_cg_jnt_implicit_'
    
# sacroiliac joint
lsj = i_joint_tag + 'lsi'
rsj = i_joint_tag + 'rsi'

# pubic joint
pj  = i_joint_tag + 'ps'

# sacroiliac and pubic joint paths
i_lsj_path = str((i_dir/ lsj).with_suffix(i_format).resolve())
i_rsj_path = str((i_dir/ rsj).with_suffix(i_format).resolve())
i_pj_path  = str((i_dir/ pj) .with_suffix(i_format).resolve())

# hip joint with gap
lhj_ac_wg = i_joint_tag + 'lh_ac_wg' 
lhj_fc_wg = i_joint_tag + 'lh_fc_wg' 
rhj_ac_wg = i_joint_tag + 'rh_ac_wg' 
rhj_fc_wg = i_joint_tag + 'rh_fc_wg'  

# hip joint without gap
lhj_ac_wog = i_joint_tag + 'lh_ac_wog' 
lhj_fc_wog = i_joint_tag + 'lh_fc_wog'
rhj_ac_wog = i_joint_tag + 'rh_ac_wog'
rhj_fc_wog = i_joint_tag + 'rh_fc_wog'

# hip joint w/wo gap paths
i_lhj_ac_wg_path = str((i_dir/ lhj_ac_wg).with_suffix(i_format).resolve())
i_rhj_ac_wg_path = str((i_dir/ rhj_ac_wg).with_suffix(i_format).resolve())
i_lhj_fc_wg_path = str((i_dir/ lhj_fc_wg).with_suffix(i_format).resolve())
i_rhj_fc_wg_path = str((i_dir/ rhj_fc_wg).with_suffix(i_format).resolve())

i_lhj_ac_wog_path = str((i_dir/ lhj_ac_wog).with_suffix(i_format).resolve())
i_rhj_ac_wog_path = str((i_dir/ rhj_ac_wog).with_suffix(i_format).resolve())
i_lhj_fc_wog_path = str((i_dir/ lhj_fc_wog).with_suffix(i_format).resolve())
i_rhj_fc_wog_path = str((i_dir/ rhj_fc_wog).with_suffix(i_format).resolve())

In [6]:
# ftetwild mid output directories - with gap
ftet_o_tag = model_id + '_csg_'

ftet_o_pg_wg   = ftet_o_tag + '_wg_pg'
ftet_o_legL_wg = ftet_o_tag + '_wg_legL' 
ftet_o_legR_wg = ftet_o_tag + '_wg_legR'

ftet_o_pg_wg_path   = str((ftet_o_dir/ ftet_o_pg_wg).resolve())
ftet_o_legL_wg_path = str((ftet_o_dir/ ftet_o_legL_wg).resolve())
ftet_o_legR_wg_path = str((ftet_o_dir/ ftet_o_legR_wg).resolve())

# ftetwild mid output directories - without gap
ftet_o_pg_wog   = ftet_o_tag + '_wog_pg'
ftet_o_legL_wog = ftet_o_tag + '_wog_legL' 
ftet_o_legR_wog = ftet_o_tag + '_wog_legR'

ftet_o_pg_wog_path   = str((ftet_o_dir/ ftet_o_pg_wog).resolve())
ftet_o_legL_wog_path = str((ftet_o_dir/ ftet_o_legL_wog).resolve())
ftet_o_legR_wog_path =  str((ftet_o_dir/ ftet_o_legR_wog).resolve())

In [7]:
# volgen output directories - with gap
o_pg_wg   = model_id + '_wg_pg'   
o_legL_wg = model_id + '_wg_legL' 
o_legR_wg = model_id + '_wg_legR'

o_pg_wg_path   = str(o_dir/ o_pg_wg)
o_legL_wg_path = str(o_dir/ o_legL_wg)
o_legR_wg_path = str(o_dir/ o_legR_wg)

nodal_pg_wg_path   = str(nodal_o_dir/ o_pg_wg)
nodal_legL_wg_path = str(nodal_o_dir/ o_legL_wg)
nodal_legR_wg_path = str(nodal_o_dir/ o_legR_wg)

# volgen output directories - without gap
o_pg_wog   = model_id + '_wog_pg'   
o_legL_wog = model_id + '_wog_legL' 
o_legR_wog = model_id + '_wog_legR'

o_pg_wog_path   = str(o_dir/ o_pg_wog)
o_legL_wog_path = str(o_dir/ o_legL_wog)
o_legR_wog_path = str(o_dir/ o_legR_wog)

nodal_pg_wog_path   = str(nodal_o_dir/ o_pg_wog)
nodal_legL_wog_path = str(nodal_o_dir/ o_legL_wog)
nodal_legR_wog_path = str(nodal_o_dir/ o_legR_wog)

VOLUME MESH GENERATION PROPERTIES:
- The volume mesh properties for an optimal mesh size is read from the subject-specific config files. 
- eps: the envelope of size epsilon. Using smaller envelope preserves features better but also takes longer time. 
  The default value of epsilon is b/1000, where b is the length of the diagonal of the bounding box. 
  Please refer to https://github.com/wildmeshing/fTetWild for more information.
- l_* : the ideal edge length. Using smaller ideal edge length gives a denser mesh but also takes longer time. 
  The default ideal edge length is b/20.
  Please refer to https://github.com/wildmeshing/fTetWild for more information.

In [8]:
# epsilon and edge length for the femur (l_leg) and the sacrum and the hip (l_girdle)
config_vol = config.vol_var

eps        = str(config_vol.epsilon)
l_leg      = str(config_vol.edge_length_leg)
l_girdle   = str(config_vol.edge_length_girdle)

# read surface mesh

In [9]:
s_vertices, s_faces   = src.read (i_sacrum_path, i_dim) 
lp_vertices, lp_faces = src.read (i_lpelvis_path, i_dim)
rp_vertices, rp_faces = src.read (i_rpelvis_path, i_dim) 
lf_vertices, lf_faces = src.read (i_lfemur_path, i_dim)
rf_vertices, rf_faces = src.read (i_rfemur_path, i_dim) 

lsj_vertices, lsj_faces = src.read (i_lsj_path, i_dim)
rsj_vertices, rsj_faces = src.read (i_rsj_path, i_dim)
pj_vertices,  pj_faces  = src.read (i_pj_path,  i_dim)

# with gap
lhj_ac_wg_vertices, lhj_ac_wg_faces = src.read (i_lhj_ac_wg_path, i_dim )
rhj_ac_wg_vertices, rhj_ac_wg_faces = src.read (i_rhj_ac_wg_path, i_dim )

lhj_fc_wg_vertices, lhj_fc_wg_faces = src.read (i_lhj_fc_wg_path, i_dim )
rhj_fc_wg_vertices, rhj_fc_wg_faces = src.read (i_rhj_fc_wg_path,i_dim )

# without gap
lhj_ac_wog_vertices, lhj_ac_wog_faces = src.read (i_lhj_ac_wog_path, i_dim )
rhj_ac_wog_vertices, rhj_ac_wog_faces = src.read (i_rhj_ac_wog_path, i_dim )

lhj_fc_wog_vertices, lhj_fc_wog_faces = src.read (i_lhj_fc_wog_path, i_dim )
rhj_fc_wog_vertices, rhj_fc_wog_faces = src.read (i_rhj_fc_wog_path,i_dim )

number of faces after reading 44510
number of faces after reading 36084
number of faces after reading 35832
number of faces after reading 45274
number of faces after reading 47032
number of faces after reading 4076
number of faces after reading 3528
number of faces after reading 468
number of faces after reading 2848
number of faces after reading 3038
number of faces after reading 17606
number of faces after reading 17756
number of faces after reading 2848
number of faces after reading 3038
number of faces after reading 17606
number of faces after reading 17756


In [10]:
frame = mp.plot( s_vertices, s_faces, c = src.bone, shading = src.sh_false )
frame.add_mesh ( lp_vertices, lp_faces, c = src.bone, shading = src.sh_false )
frame.add_mesh ( rp_vertices, rp_faces, c = src.bone, shading = src.sh_false )
frame.add_mesh ( lf_vertices, lf_faces, c = src.bone, shading = src.sh_false )
frame.add_mesh ( rf_vertices, rf_faces, c = src.bone, shading = src.sh_false )
frame.add_mesh ( lhj_ac_wg_vertices, lhj_ac_wg_faces, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( lhj_fc_wg_vertices, lhj_fc_wg_faces, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( rhj_ac_wg_vertices, rhj_ac_wg_faces, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( rhj_fc_wg_vertices, rhj_fc_wg_faces, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( lsj_vertices, lsj_faces, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( rsj_vertices, rsj_faces, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( pj_vertices, pj_faces, c = src.pastel_light_blue, shading = src.sh_false)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(8.5043048…

11

# volume mesh generation -  with a small gap in the hip joint (wg)

- First, we make a union boolean operation function to glue the sub-domains together.
 - pelvic girdle: The boolean boolean operation glues the sacrum, the paired pelvic bones, the paired sacroiliac jooints, and the pubic symphysis
 - left leg: The boolean boolean operation glues the left femoral bone and the left femoral cartilage together.
 - right leg: The boolean boolean operation glues the right femoral bone and the right femoral cartilage together.
- Second, we create volume mesh for the sub-domains simultaneously rather than building our discretized model one by one. fTetWild constructs volume meshes inside and outside the model, filling a bounding box around the model.
- Third, We apply a post-processing step to extract the interior volume of each object and filter out the elements that do not belong to any of the objects.

1- PELVIC GIRDLE (WG)

In [11]:
# csg operation and json file
op_0      = src.mk_union (i_lpelvis_path, i_lhj_ac_wg_path)
op_1      = src.mk_union (i_rpelvis_path, i_rhj_ac_wg_path)
op_2      = src.mk_union (op_0, i_lsj_path)
op_3      = src.mk_union (op_1, i_rsj_path)
op_4      = src.mk_union (op_2, op_3)
op_5      = src.mk_union (op_4, i_sacrum_path)
op_girdle = src.mk_union (op_5, i_pj_path)

json_pg_wg_path = src.mk_json(ftet_o_pg_wg, op_girdle, json_o_dir)

# volume mesh with no inside/out classification
src.run_boolean(ftetwild_dir, json_pg_wg_path, ftet_o_pg_wg_path, eps, l_girdle)

# post-processing
src.girdle_filter (ftet_o_pg_wg_path, o_pg_wg_path, nodal_pg_wg_path,
                   lsj_vertices, lsj_faces, rsj_vertices, rsj_faces, 
                   lhj_ac_wg_vertices, lhj_ac_wg_faces, rhj_ac_wg_vertices, rhj_ac_wg_faces,
                   pj_vertices, pj_faces, s_vertices, s_faces,
                   lp_vertices, lp_faces, rp_vertices, rp_faces,
                   i_dim, o_dim)


[1 2 3 4 5 6 7 8]
length of elements one 4613
length of elements two 2359
length of elements three 1839
length of elements four 1792
length of elements five 454
length of elements six 48470
length of elements seven 39798
length of elements eight 39968
length of labels 139293


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(8.8991317…

2- LEFT LEG (WG)

In [12]:
# csg operation and json file
op_legL = src.mk_union (i_lfemur_path, i_lhj_fc_wg_path)

json_legL_wg_path = src.mk_json (ftet_o_legL_wg, op_legL, json_o_dir)

# volume mesh with no inside/out classification
src.run_boolean (ftetwild_dir, json_legL_wg_path, ftet_o_legL_wg_path, eps, l_leg)

# post-processing
src.leg_filter (ftet_o_legL_wg_path, o_legL_wg_path, nodal_legL_wg_path,
                lhj_fc_wg_vertices, lhj_fc_wg_faces,
                lf_vertices, lf_faces,
                i_dim, o_dim)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(133.58573…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(133.59288…

number of physical elements [1 2]
length of elements 1 9387
length of elements 2 102175
length of labels 111562


3- RIGHT LEG (WG)

In [13]:
# csg operation and json file
op_legR = src.mk_union(i_rfemur_path, i_rhj_fc_wg_path)

json_legR_wg_path = src.mk_json(ftet_o_legR_wg, op_legR, json_o_dir)

# volume mesh with no inside/out classification
src.run_boolean (ftetwild_dir, json_legR_wg_path, ftet_o_legR_wg_path, eps, l_leg)

# post-processing
src.leg_filter (ftet_o_legR_wg_path, o_legR_wg_path, nodal_legR_wg_path,
                rhj_fc_wg_vertices, rhj_fc_wg_faces,
                rf_vertices, rf_faces,
                i_dim, o_dim)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-111.5471…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-111.5410…

number of physical elements [1 2]
length of elements 1 8881
length of elements 2 103126
length of labels 112007


# volume mesh generation - without gap in the hip joint (wog)

1- PELVIC GIRDLE (WOG)

In [14]:
# csg operation and json file
op_0      = src.mk_union (i_lpelvis_path, i_lhj_ac_wog_path)
op_1      = src.mk_union (i_rpelvis_path, i_rhj_ac_wog_path)
op_2      = src.mk_union (op_0, i_lsj_path)
op_3      = src.mk_union (op_1, i_rsj_path)
op_4      = src.mk_union (op_2, op_3)
op_5      = src.mk_union (op_4, i_sacrum_path)
op_girdle = src.mk_union (op_5, i_pj_path)

json_pg_wog_path = src.mk_json(ftet_o_pg_wog, op_girdle, json_o_dir)

# volume mesh with no inside/out classification
src.run_boolean(ftetwild_dir, json_pg_wog_path, ftet_o_pg_wog_path, eps, l_girdle)

# post-processing
src.girdle_filter (ftet_o_pg_wog_path, o_pg_wog_path, nodal_pg_wog_path,
                   lsj_vertices, lsj_faces, rsj_vertices, rsj_faces, 
                   lhj_ac_wog_vertices, lhj_ac_wog_faces, rhj_ac_wog_vertices, rhj_ac_wog_faces,
                   pj_vertices, pj_faces, s_vertices, s_faces,
                   lp_vertices, lp_faces, rp_vertices, rp_faces,
                   i_dim, o_dim)

[1 2 3 4 5 6 7 8]
length of elements one 4245
length of elements two 2152
length of elements three 1765
length of elements four 1817
length of elements five 397
length of elements six 46793
length of elements seven 39880
length of elements eight 38252
length of labels 135301


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(8.9270019…

2- LEFT LEG (WOG)

In [15]:
# csg operation and json file
op_legL = src.mk_union (i_lfemur_path, i_lhj_fc_wog_path)

json_legL_wog_path = src.mk_json(ftet_o_legL_wog, op_legL, json_o_dir)

# volume mesh with no inside/out classification
src.run_boolean (ftetwild_dir, json_legL_wog_path, ftet_o_legL_wog_path, eps, l_leg)

# post-processing
src.leg_filter (ftet_o_legL_wog_path, o_legL_wog_path, nodal_legL_wog_path,
                lhj_fc_wog_vertices, lhj_fc_wog_faces,
                lf_vertices, lf_faces,
                i_dim, o_dim)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(133.50089…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(133.50408…

number of physical elements [1 2]
length of elements 1 9446
length of elements 2 100597
length of labels 110043


3- RIGHT LEG (WOG)

In [16]:
# csg operation and json file 
op_legR = src.mk_union(i_rfemur_path, i_rhj_fc_wog_path)

json_legR_wog_path = src.mk_json(ftet_o_legR_wog, op_legR, json_o_dir)

# volume mesh with no inside/out classification
src.run_boolean (ftetwild_dir, json_legR_wog_path, ftet_o_legR_wog_path, eps, l_leg)

# post-processing
src.leg_filter (ftet_o_legR_wog_path, o_legR_wog_path, nodal_legR_wog_path,
                rhj_fc_wog_vertices, rhj_fc_wog_faces,
                rf_vertices, rf_faces,
                i_dim, o_dim )

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-111.4756…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-111.4657…

number of physical elements [1 2]
length of elements 1 8953
length of elements 2 100970
length of labels 109923


# make sure of the gaps

In [17]:
# # with gap
# lfc_wg_verices, lfc_wg_faces = src.read(o_legL_wg_path + '_jnt_fc.obj', i_dim)
# lpc_wg_verices, lpc_wg_faces = src.read(o_pg_wg_path   + '_jnt_lac.obj', i_dim)
# rfc_wg_verices, rfc_wg_faces = src.read(o_legR_wg_path + '_jnt_fc.obj', i_dim)
# rpc_wg_verices, rpc_wg_faces = src.read(o_pg_wg_path   + '_jnt_rac.obj', i_dim)

# lsd_wg, _, _ = igl.signed_distance(lfc_wg_verices,lpc_wg_verices, lpc_wg_faces, return_normals=False)
# rsd_wg, _, _ = igl.signed_distance(rfc_wg_verices,rpc_wg_verices, rpc_wg_faces, return_normals=False)

# print('')
# print('minimum distance in the left hip joint with gap:',np.round(np.min(lsd_wg),2))
# print('minimum distance in the right hip joint with gap:',np.round(np.min(rsd_wg),2))
# print('')

# # without gap
# lfc_wog_verices, lfc_wog_faces = src.read(o_legL_wog_path + '_jnt_fc.obj', i_dim)
# lpc_wog_verices, lpc_wog_faces = src.read(o_pg_wog_path   + '_jnt_lac.obj', i_dim)
# rfc_wog_verices, rfc_wog_faces = src.read(o_legR_wog_path + '_jnt_fc.obj', i_dim)
# rpc_wog_verices, rpc_wog_faces = src.read(o_pg_wog_path   + '_jnt_rac.obj', i_dim)

# lsd_wog, _, _ = igl.signed_distance(lfc_wog_verices,lpc_wog_verices, lpc_wog_faces, return_normals=False)
# rsd_wog, _, _ = igl.signed_distance(rfc_wog_verices,rpc_wog_verices, rpc_wog_faces, return_normals=False)

# print('')
# print('minimum distance in the left hip joint without gap:',np.round(np.min(lsd_wog),2))
# print('minimum distance in the right hip joint without gap:',np.round(np.min(rsd_wog),2))
# print('')

number of faces after reading 6568
number of faces after reading 1452
number of faces after reading 6246
number of faces after reading 1410

minimum distance in the left hip joint with gap: 0.15
minimum distance in the right hip joint with gap: 0.08

number of faces after reading 6524
number of faces after reading 1396
number of faces after reading 6282
number of faces after reading 1418

minimum distance in the left hip joint without gap: -0.11
minimum distance in the right hip joint without gap: -0.11

